In [1]:
import pandas as pd
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import models
from tensorflow.keras import layers, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Bidirectional, LSTM
from tensorflow.keras.optimizers import SGD
from tensorflow import keras
import numpy as np

In [2]:
pickle_in1 = open('../Twitter_bot_detection_713/data/pickled_data/X_train_embed_25.pickle', "rb")
X_train_embed = pickle.load(pickle_in1)
pickle_in2 = open("../Twitter_bot_detection_713/data/pickled_data/X_test_embed_25.pickle", "rb")
X_test_embed = pickle.load(pickle_in2)
pickle_in3 = open("../Twitter_bot_detection_713/data/pickled_data/y_train.pickle", "rb")
y_train = pickle.load(pickle_in3)
pickle_in4= open("../Twitter_bot_detection_713/data/pickled_data/y_test.pickle", "rb")
y_test = pickle.load(pickle_in4)

In [3]:
X_test_pad = pad_sequences(X_test_embed, dtype='float32',padding='post',maxlen=60)
X_train_pad = pad_sequences(X_train_embed, dtype='float32',padding='post',maxlen=60)

In [4]:
opt = keras.optimizers.SGD(learning_rate=0.01,momentum=0.9)
def initialize_model_rnn2():

    model = Sequential()

    model.add(layers.Masking(mask_value=0.0, input_shape=(60, 25)))

    model.add(Bidirectional(LSTM(200, return_sequences=True)))
    
    model.add(Bidirectional(LSTM(200, return_sequences=True)))
    
    model.add(Bidirectional(LSTM(200, return_sequences=False)))

    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy', 'Precision', 'Recall'])
    return model

In [5]:
text_model = initialize_model_rnn2()

2021-11-30 22:11:42.727482: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
es = EarlyStopping(patience=5, restore_best_weights=True)
history = text_model.fit(X_train_pad,
                                    y_train,
                                    epochs=15,
                                    batch_size=128,
                                    validation_split=0.3,
                                    callbacks=[es])

2021-11-30 22:11:48.558205: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/15
 465/7057 [>.............................] - ETA: 1:31:55 - loss: 0.3961 - accuracy: 0.8648 - precision: 0.3000 - recall: 7.4608e-04

KeyboardInterrupt: 